In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx
import dbase_tools

# Import API key
from api_keys import api_key
single_city_url = "http://api.openweathermap.org/data/2.5/weather?"

## Bokeh imports

In [2]:
#%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import DatetimeTickFormatter
##import pandas as pd
#import numpy as np
from bokeh.models.tools import HoverTool
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [5]:
city_name = "portland"
city_id = "5746545"
city_units = "metric"
covid = "New_York_Times/covid-19-state-level-data.csv"

In [6]:
# Create City/location and transport Classes
# ----------------------------------

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


# Creates Classes which will serve as the anchor points for our Tables
class State(Base):
    __tablename__ = 'location'
    id = Column(Integer, primary_key=True)
    city_name = Column(String(255))
    date = Column(String(255))
    cases = Column(Integer)
    deaths = Column(Integer)
    new_cases = Column(Integer)
    new_deaths = Column(Integer)

In [7]:
covid_df = pd.read_csv(covid)

In [8]:
covid_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
1824,2020-04-04,Virginia,51,2407,52
1825,2020-04-04,Washington,53,7498,318
1826,2020-04-04,West Virginia,54,282,2
1827,2020-04-04,Wisconsin,55,2112,60


## data cleaning

In [9]:
covcopy = covid_df.copy()

In [10]:
state_list = covid_df['state'].unique()

In [11]:
state_list

array(['Washington', 'Illinois', 'California', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon', 'Florida',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'West Virginia', 'Northern Mariana Islands'], dtype=object)

## Isolate regions by state

In [12]:
test_module = dbase_tools.group_states(covid_df,state_list)

In [13]:
test_module.head()

,date,state,fips,cases,deaths,new_deaths,new_cases
0,2020-01-21,Washington,53,1,0,0,1
1,2020-01-22,Washington,53,1,0,0,0
2,2020-01-23,Washington,53,1,0,0,0
3,2020-01-24,Illinois,17,1,0,0,0
4,2020-01-24,Washington,53,1,0,0,0


In [14]:
region_list_country = covid_df.loc[covid_df['state'] == "New York"]

In [15]:
region_list_country.head(5)

,date,state,fips,cases,deaths,new_deaths,new_cases
246,2020-03-01,New York,36,1,0,0,0
261,2020-03-02,New York,36,1,0,0,1
276,2020-03-03,New York,36,2,0,4,92
293,2020-03-04,New York,36,11,0,0,0
313,2020-03-05,New York,36,22,0,0,0


### start here for state_data_sort function.

In [16]:
cases_state = [case for case in region_list_country['cases']]

In [17]:
new_case = []
for day in range(0,len(cases_state)):
    if day == 0:
        new_case.append(cases_state[day])
    elif cases_state[day] > cases_state[day - 1]:
        new_case.append(cases_state[day] - cases_state[day - 1])
    else:
        new_case.append(0)

In [18]:
region_list_country['new_cases'] = new_case

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
deaths_state = [case for case in region_list_country['deaths']]

In [20]:
new_deaths = []
for day in range(0,len(deaths_state)):
    if day == 0:
        new_deaths.append(deaths_state[day])
    elif deaths_state[day] > deaths_state[day - 1]:
        new_deaths.append(deaths_state[day] - deaths_state[day - 1])
    else:
        new_deaths.append(0)

In [21]:
region_list_country['new_deaths'] = new_deaths

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
region_list_country.head(5)

,date,state,fips,cases,deaths,new_deaths,new_cases
246,2020-03-01,New York,36,1,0,0,1
261,2020-03-02,New York,36,1,0,0,0
276,2020-03-03,New York,36,2,0,0,1
293,2020-03-04,New York,36,11,0,0,9
313,2020-03-05,New York,36,22,0,0,11


In [23]:
region_list_country.to_csv("new_york_data.csv")

In [24]:
region_list_country.to_json("new_york_data.json")

# Create object to be stored in database

In [25]:
state_base = []
for i,row in region_list_country.iterrows():
    state_base.append(State(city_name = row['state'],date = row['date'], cases=row['cases'],deaths=row['deaths'],
                      new_cases=row['new_cases'],new_deaths=row['new_deaths']))

In [26]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///coviddata.sqlite")
conn = engine.connect()

In [27]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [28]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [29]:
for date_data in state_base:
    session.add(date_data)
#[session.add(date_data) for date_data in state_base]

In [30]:
session.commit()

OperationalError: (sqlite3.OperationalError) table location has no column named city_name
[SQL: INSERT INTO location (city_name, date, cases, deaths, new_cases, new_deaths) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ('New York', '2020-03-01', 1, 0, 1, 0)]
(Background on this error at: http://sqlalche.me/e/e3q8)

dog_list = session.query(Dog)
for doggy in dog_list:
    print(doggy.name)

In [31]:
date_list = session.query(State)
for date in date_list:
    print(date.cases)

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) table location has no column named city_name
[SQL: INSERT INTO location (city_name, date, cases, deaths, new_cases, new_deaths) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ('New York', '2020-03-01', 1, 0, 1, 0)]
(Background on this error at: http://sqlalche.me/e/e3q8) (Background on this error at: http://sqlalche.me/e/7s2a)

In [32]:
ref_date = []

for date in region_list_country['date']:
    ref_date.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

In [43]:
ref_date

[numpy.datetime64('2020-03-01T00:00:00.000000'),
 numpy.datetime64('2020-03-02T00:00:00.000000'),
 numpy.datetime64('2020-03-03T00:00:00.000000'),
 numpy.datetime64('2020-03-04T00:00:00.000000'),
 numpy.datetime64('2020-03-05T00:00:00.000000'),
 numpy.datetime64('2020-03-06T00:00:00.000000'),
 numpy.datetime64('2020-03-07T00:00:00.000000'),
 numpy.datetime64('2020-03-08T00:00:00.000000'),
 numpy.datetime64('2020-03-09T00:00:00.000000'),
 numpy.datetime64('2020-03-10T00:00:00.000000'),
 numpy.datetime64('2020-03-11T00:00:00.000000'),
 numpy.datetime64('2020-03-12T00:00:00.000000'),
 numpy.datetime64('2020-03-13T00:00:00.000000'),
 numpy.datetime64('2020-03-14T00:00:00.000000'),
 numpy.datetime64('2020-03-15T00:00:00.000000'),
 numpy.datetime64('2020-03-16T00:00:00.000000'),
 numpy.datetime64('2020-03-17T00:00:00.000000'),
 numpy.datetime64('2020-03-18T00:00:00.000000'),
 numpy.datetime64('2020-03-19T00:00:00.000000'),
 numpy.datetime64('2020-03-20T00:00:00.000000'),
 numpy.datetime64('2

date_groups = []

for date in indy_location_moving[0]['date']:
    date_groups_1.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

In [33]:
date_cases = []

for date in ref_date:
    date_cases.append(str(date))

# pass plot data as well as hovertool data in to plot.
source_cases = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['cases'],
            desc=date_cases,
        )
    )

In [34]:
# pass plot data as well as hovertool data in to plot.
source_deaths = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['deaths'],
            desc=date_cases,
        )
    )

In [35]:
# pass plot data as well as hovertool data in to plot.
source_new_deaths = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['new_deaths'],
            desc=date_cases,
        )
    )

In [36]:
# pass plot data as well as hovertool data in to plot.
source_new_cases = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=region_list_country['new_cases'],
            desc=date_cases,
        )
    )

In [37]:
# configure plot tools including the hover tool / tooltips.  Displays date / time of individual data points when hovering 
#over individual points.
TOOLSCCD = [HoverTool(tooltips=[
            ("cvd_date", "@desc"),("cases","$y")],
                  formatters={'DateTime': 'datetime'}),'wheel_zoom','save','crosshair','pan','reset']

In [38]:
cid = figure(title = f"Values for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cid.xaxis.axis_label = 'Event Date'
cid.yaxis.axis_label = 'COVID Total Cases'
# Setup x axis for using Datetime formatting.
cid.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cid.xaxis.major_label_orientation = pi/4
cid.circle('x', 'y', line_color="navy", fill_color="red", fill_alpha=0.2, size=10,source=source_cases)


GlyphRenderer(id='1045', ...)

In [39]:
cix = figure(title = f"Deaths for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cix.xaxis.axis_label = 'Event Date'
cix.yaxis.axis_label = 'COVID Total Deaths'
# Setup x axis for using Datetime formatting.
cix.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cix.xaxis.major_label_orientation = pi/4
cix.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_deaths)


GlyphRenderer(id='1087', ...)

In [40]:
cin = figure(title = f"New Deaths for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cin.xaxis.axis_label = 'Event Date'
cin.yaxis.axis_label = 'COVID New Deaths'
# Setup x axis for using Datetime formatting.
cin.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cin.xaxis.major_label_orientation = pi/4
cin.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_new_deaths)

GlyphRenderer(id='1129', ...)

In [41]:
cinc = figure(title = f"New Cases for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cinc.xaxis.axis_label = 'Event Date'
cinc.yaxis.axis_label = 'COVID New Cases'
# Setup x axis for using Datetime formatting.
cinc.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cinc.xaxis.major_label_orientation = pi/4
cinc.circle('x', 'y', line_color="navy", fill_color="yellow", fill_alpha=0.2, size=10,source=source_new_cases)

GlyphRenderer(id='1171', ...)

In [42]:
show(cid)

In [30]:
show(cix)

In [31]:
show(cin)

In [32]:
show(cinc)